In [1]:
import duckdb
#import json

### Repair JSON input

In [2]:
valid_entries = []
with open("nextcloud.log.json", "r", encoding="utf-8") as f:
    for line in f:
        try:
            # Process each line as a JSON object
            valid_entries.append(json.loads(line))  
        except json.JSONDecodeError:
            print("Skipping malformed line")

# Save the repaired JSON
with open("fixed_nextcloud.log.json", "w", encoding="utf-8") as f:
    json.dump(valid_entries, f, separators=(',', ':'))

### Ingest and group data

In [36]:
%%time

json_input = duckdb.read_json("fixed_nextcloud.log.json")

domain_errors = duckdb.sql("""
                       SELECT *
                       FROM json_input
                       WHERE message LIKE '%Trusted domain error%'
                       -- LIMIT 10
                           """)

gby_user_agent = duckdb.sql("""
                        SELECT CAST(remoteAddr AS INET) AS remoteAddr_inet,
                        COUNT(*) AS count
                        FROM domain_errors
                        WHERE userAgent LIKE '%python-requests%'
                        GROUP BY remoteAddr
                        ORDER BY count DESC
                             """)

print(gby_user_agent)

┌─────────────────┬───────┐
│ remoteAddr_inet │ count │
│      inet       │ int64 │
├─────────────────┼───────┤
│ 115.159.220.67  │  1610 │
│ 91.250.240.220  │   166 │
│ 20.163.106.32   │   148 │
│ 94.102.61.7     │   116 │
│ 198.98.183.135  │    90 │
│ 185.242.226.109 │    67 │
│ 185.242.226.88  │    54 │
│ 34.140.248.32   │    38 │
│ 130.211.54.158  │    38 │
│ 34.77.127.183   │    37 │
│      ·          │     · │
│      ·          │     · │
│      ·          │     · │
│ 61.6.203.162    │     1 │
│ 172.177.94.54   │     1 │
│ 147.78.47.90    │     1 │
│ 54.237.219.112  │     1 │
│ 47.245.112.112  │     1 │
│ 34.140.124.166  │     1 │
│ 149.18.73.140   │     1 │
│ 34.140.65.158   │     1 │
│ 88.215.26.222   │     1 │
│ 35.241.185.104  │     1 │
├─────────────────┴───────┤
│   319 rows (20 shown)   │
└─────────────────────────┘

CPU times: user 4.89 s, sys: 397 ms, total: 5.29 s
Wall time: 5.28 s


In [32]:
# domain_errors = duckdb.sql("""
#                        SELECT time, remoteAddr, userAgent
#                        FROM json_input
#                        WHERE message LIKE '%Trusted domain error%'
#                        AND userAgent LIKE '%bot%'
#                        --- LIMIT 10
#                            """)

# print(domain_errors)

In [33]:
# login_errors = duckdb.sql("""
#                        SELECT time, remoteAddr, message, 
#                        FROM json_input
#                        WHERE message LIKE '%Login failed%'
#                        AND message NOT LIKE '%vnegi10%'
#                        AND message NOT LIKE '%mdash%'
#                        --- LIMIT 10
#                            """)

# print(login_errors)

### Read IP database and join to group

In [37]:
%%time

ipv4_city = duckdb.sql("""
                SELECT * 
                FROM read_csv('dbip-city-lite-2025-02.csv',
                               columns = {
                               'ip_start': 'VARCHAR(15)',
                               'ip_end': 'VARCHAR(15)', 
                               'continent': 'VARCHAR(2)',
                               'country': 'VARCHAR(2)',
                               'stateprov': 'TEXT',
                               'city': 'TEXT',
                               'latitude': 'FLOAT',
                               'longitude': 'FLOAT'
                               }, 
                               header = False,
                               ignore_errors = true)
                WHERE ip_start LIKE '%.%'
                      """)

ipv4_city_subset = duckdb.sql("""
                        SELECT CAST(ip_start as INET) AS ip_start_inet, 
                               CAST(ip_end as INET) AS ip_end_inet,
                               country,
                               stateprov,
                               city
                        FROM ipv4_city
                        -- LIMIT 10
                            """)

gby_user_agent_city_join = duckdb.sql("""
                                SELECT gua.remoteAddr_inet,
                                       gua.count,
                                       ics.country,
                                       ics.stateprov,
                                       ics.city
                                FROM gby_user_agent gua
                                JOIN ipv4_city_subset ics
                                ON gua.remoteAddr_inet
                                BETWEEN ics.ip_start_inet AND ics.ip_end_inet
                                ORDER BY count DESC
                                """)

print(gby_user_agent_city_join)

┌─────────────────┬───────┬─────────┬──────────────────┬─────────────────────────────────────┐
│ remoteAddr_inet │ count │ country │    stateprov     │                city                 │
│      inet       │ int64 │ varchar │     varchar      │               varchar               │
├─────────────────┼───────┼─────────┼──────────────────┼─────────────────────────────────────┤
│ 115.159.220.67  │  1610 │ CN      │ Shanghai         │ Shanghai                            │
│ 91.250.240.220  │   166 │ PT      │ Lisbon           │ Lisbon                              │
│ 20.163.106.32   │   148 │ US      │ Arizona          │ Phoenix                             │
│ 94.102.61.7     │   116 │ NL      │ North Holland    │ Amsterdam                           │
│ 185.242.226.109 │    67 │ NL      │ North Holland    │ Amsterdam                           │
│ 185.242.226.88  │    54 │ NL      │ North Holland    │ Amsterdam                           │
│ 130.211.54.158  │    38 │ BE      │ Brussels Cap

### Read ASN database and join to group

In [38]:
asn = duckdb.sql("""
                SELECT * 
                FROM read_csv('dbip-asn-lite-2025-02.csv',
                               columns = {
                               'ip_start': 'VARCHAR(15)',
                               'ip_end': 'VARCHAR(15)',
                               'as_number': 'INT',
                               'as_org': 'TEXT'                              
                               }, 
                               header = False,
                               ignore_errors = true)
                WHERE ip_start LIKE '%.%'
                      """)

asn_subset = duckdb.sql("""
                        SELECT CAST(ip_start as INET) AS ip_start_inet, 
                               CAST(ip_end as INET) AS ip_end_inet,
                               as_org,
                        FROM asn
                        -- LIMIT 10
                            """)

gby_user_agent_city_asn_join = duckdb.sql("""
                                SELECT gua.remoteAddr_inet,
                                       gua.count,
                                       gua.country,
                                       gua.stateprov,
                                       gua.city,
                                       asn.as_org
                                FROM gby_user_agent_city_join gua
                                JOIN asn_subset asn
                                ON gua.remoteAddr_inet
                                BETWEEN asn.ip_start_inet AND asn.ip_end_inet
                                ORDER BY count DESC
                                """)

print(gby_user_agent_city_asn_join)

┌─────────────────┬───────┬─────────┬──────────────────┬────────────┬───────────────────────────────────────────────────┐
│ remoteAddr_inet │ count │ country │    stateprov     │    city    │                      as_org                       │
│      inet       │ int64 │ varchar │     varchar      │  varchar   │                      varchar                      │
├─────────────────┼───────┼─────────┼──────────────────┼────────────┼───────────────────────────────────────────────────┤
│ 115.159.220.67  │  1610 │ CN      │ Shanghai         │ Shanghai   │ Shenzhen Tencent Computer Systems Company Limited │
│ 91.250.240.220  │   166 │ PT      │ Lisbon           │ Lisbon     │ HostRoyale Technologies Pvt Ltd                   │
│ 20.163.106.32   │   148 │ US      │ Arizona          │ Phoenix    │ Microsoft Corporation                             │
│ 94.102.61.7     │   116 │ NL      │ North Holland    │ Amsterdam  │ IP Volume inc                                     │
│ 185.242.226.109 │    6

### Performance - Full query

In [40]:
%%time

json_input = duckdb.read_json("fixed_nextcloud.log.json")

domain_errors = duckdb.sql("""
                       SELECT *
                       FROM json_input
                       WHERE message LIKE '%Trusted domain error%'
                       -- LIMIT 10
                           """)

gby_user_agent = duckdb.sql("""
                        SELECT CAST(remoteAddr AS INET) AS remoteAddr_inet,
                        COUNT(*) AS count
                        FROM domain_errors
                        WHERE userAgent LIKE '%Macintosh%'
                        GROUP BY remoteAddr
                        ORDER BY count DESC
                             """)

ipv4_city = duckdb.sql("""
                SELECT * 
                FROM read_csv('dbip-city-lite-2025-02.csv',
                               columns = {
                               'ip_start': 'VARCHAR(15)',
                               'ip_end': 'VARCHAR(15)', 
                               'continent': 'VARCHAR(2)',
                               'country': 'VARCHAR(2)',
                               'stateprov': 'TEXT',
                               'city': 'TEXT',
                               'latitude': 'FLOAT',
                               'longitude': 'FLOAT'
                               }, 
                               header = False,
                               ignore_errors = true)
                WHERE ip_start LIKE '%.%'
                      """)

ipv4_city_subset = duckdb.sql("""
                        SELECT CAST(ip_start as INET) AS ip_start_inet, 
                               CAST(ip_end as INET) AS ip_end_inet,
                               country,
                               stateprov,
                               city
                        FROM ipv4_city
                        -- LIMIT 10
                            """)

gby_user_agent_city_join = duckdb.sql("""
                                SELECT gua.remoteAddr_inet,
                                       gua.count,
                                       ics.country,
                                       ics.stateprov,
                                       ics.city
                                FROM gby_user_agent gua
                                JOIN ipv4_city_subset ics
                                ON gua.remoteAddr_inet
                                BETWEEN ics.ip_start_inet AND ics.ip_end_inet
                                ORDER BY count DESC
                                """)

asn = duckdb.sql("""
                SELECT * 
                FROM read_csv('dbip-asn-lite-2025-02.csv',
                               columns = {
                               'ip_start': 'VARCHAR(15)',
                               'ip_end': 'VARCHAR(15)',
                               'as_number': 'INT',
                               'as_org': 'TEXT'                              
                               }, 
                               header = False,
                               ignore_errors = true)
                WHERE ip_start LIKE '%.%'
                      """)

asn_subset = duckdb.sql("""
                        SELECT CAST(ip_start as INET) AS ip_start_inet, 
                               CAST(ip_end as INET) AS ip_end_inet,
                               as_org,
                        FROM asn
                        -- LIMIT 10
                            """)

gby_user_agent_city_asn_join = duckdb.sql("""
                                SELECT gua.remoteAddr_inet,
                                       gua.count,
                                       gua.country,
                                       gua.stateprov,
                                       gua.city,
                                       asn.as_org
                                FROM gby_user_agent_city_join gua
                                JOIN asn_subset asn
                                ON gua.remoteAddr_inet
                                BETWEEN asn.ip_start_inet AND asn.ip_end_inet
                                ORDER BY count DESC
                                """)

print(gby_user_agent_city_asn_join)

┌─────────────────┬───────┬─────────┬───────────────────────┬─────────────────────────────────────┬────────────────────────────────────────────┐
│ remoteAddr_inet │ count │ country │       stateprov       │                city                 │                   as_org                   │
│      inet       │ int64 │ varchar │        varchar        │               varchar               │                  varchar                   │
├─────────────────┼───────┼─────────┼───────────────────────┼─────────────────────────────────────┼────────────────────────────────────────────┤
│ 156.38.245.18   │  7102 │ ZA      │ Gauteng               │ Johannesburg                        │ Xneelo (Pty) Ltd                           │
│ 103.121.39.54   │  1323 │ BD      │ Chittagong            │ Chittagong (Chawk Bazar)            │ Digital Dot Net                            │
│ 3.236.57.159    │  1312 │ US      │ Virginia              │ Ashburn                             │ Amazon.com, Inc.              